In [1]:
import sys
sys.path.append("./tacotron2/") # go to parent dir
# from customFunctions import *
sys.path.append("./waveglow/")

In [2]:
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence

In [3]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                       interpolation='none')

In [4]:
hparams = create_hparams()
hparams.sampling_rate = 22050
hparams.training_files = './tacotron2/filelists/vlsp/train.txt'

attention_dim: 128
attention_location_kernel_size: 31
attention_location_n_filters: 32
attention_rnn_dim: 1024
batch_size: 16
cudnn_benchmark: false
cudnn_enabled: true
decoder_rnn_dim: 1024
dist_backend: nccl
dist_url: tcp://localhost:54321
distributed_run: false
dynamic_loss_scaling: true
encoder_embedding_dim: 512
encoder_kernel_size: 5
encoder_n_convolutions: 3
epochs: 37
filter_length: 1024
fp16_run: false
gate_threshold: 0.5
grad_clip_thresh: 1.0
hop_length: 256
ignore_layers:
- embedding.weight
iters_per_checkpoint: 200
learning_rate: 0.001
load_mel_from_disk: false
mask_padding: true
max_decoder_steps: 1000
max_wav_value: 32768.0
mel_fmax: 8000.0
mel_fmin: 0.0
n_frames_per_step: 1
n_mel_channels: 80
n_symbols: 150
p_attention_dropout: 0.1
p_decoder_dropout: 0.1
postnet_embedding_dim: 512
postnet_kernel_size: 5
postnet_n_convolutions: 5
prenet_dim: 256
sampling_rate: 22050
seed: 1234
symbols_embedding_dim: 512
text_cleaners:
- flowtron_cleaners
training_files: filelists/vlsp/tra

In [5]:
checkpoint_path = "./tacotron2/checkpoints/checkpoint_30119"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval().half()

In [17]:
from waveglow.denoiser import Denoiser
waveglow_path = './waveglow/checkpoints/waveglow_9821'
waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

In [18]:
from data_utils import TextMelLoader, TextMelCollate
trainset = TextMelLoader(hparams.training_files, hparams)

In [60]:
textx = "người ta thường hay nghĩ quá đáng * ngay cả"


In [61]:
inputs=trainset.get_text(
    textx
)
sequence = torch.autograd.Variable(
    inputs.unsqueeze(0)).cuda().long()
sequence.size()
with torch.no_grad():
    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)

In [62]:
sequence

tensor([[51, 40, 81, 66, 42,  0, 74,  6,  0, 74, 41, 81, 66, 51, 40,  0, 41,  6,
         89,  0, 51, 40, 41, 46,  0, 71, 75,  7,  0, 27,  7, 51, 40,  0,  4,  0,
         51, 40,  6, 89,  0, 25, 11,  0, 99]], device='cuda:0')

In [63]:
# mel_outputs, mel_outputs_postnet, _, alignments

In [64]:
# trainset.text_embedding.symbol2numeric_dict

In [65]:
with torch.no_grad():
    audio = waveglow.infer(z.unsqueeze(0).float().half().to("cuda"))


In [66]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [67]:
audio

tensor([[-0.0445, -0.0422, -0.0572,  ...,  0.2107,  0.2036,  0.1842]],
       device='cuda:0', dtype=torch.float16)

In [35]:
mel_outputs_postnet

tensor([[[ -8.6250,  -6.6445,  -4.9102,  ...,  -4.9141,  -5.2617,  -5.9688],
         [ -8.7812,  -7.0273,  -4.8438,  ...,  -4.8711,  -5.2109,  -6.0195],
         [ -9.2656,  -7.1953,  -4.5664,  ...,  -4.9141,  -5.3516,  -6.1250],
         ...,
         [-11.0469,  -9.8438,  -8.3125,  ...,  -8.4922,  -8.8359,  -9.0391],
         [-11.1172,  -9.9531,  -8.2656,  ...,  -8.6094,  -8.9766,  -9.1641],
         [-11.1484,  -9.8359,  -8.0781,  ...,  -8.5469,  -8.9141,  -9.1562]]],
       device='cuda:0', dtype=torch.float16)

In [38]:
z=trainset.get_mel("./tacotron2/filelists/vlsp/wavs_train/07271.wav")

In [43]:
z.unsqueeze(0).float().to("cuda")

tensor([[[ -7.6114,  -6.9149,  -4.5901,  ...,  -5.1608,  -5.0065,  -5.1246],
         [ -8.2182,  -7.3237,  -4.9716,  ...,  -5.3795,  -5.0775,  -5.3920],
         [ -8.4934,  -7.3825,  -5.1712,  ...,  -5.1540,  -5.7225,  -6.2172],
         ...,
         [-10.7848, -10.0147,  -8.4081,  ...,  -8.3630,  -8.5582,  -8.5523],
         [-11.4279, -10.3909,  -8.2990,  ...,  -8.4469,  -8.6181,  -8.7357],
         [-10.4379,  -9.7481,  -7.8345,  ...,  -8.1815,  -8.8624,  -9.2118]]],
       device='cuda:0')

In [49]:
z -mel_outputs_postnet.squeeze(0)

RuntimeError: The size of tensor a (341) must match the size of tensor b (346) at non-singleton dimension 1

In [50]:
mel_outputs_postnet.size(),z.size()

(torch.Size([1, 80, 346]), torch.Size([80, 341]))

In [56]:
torch.mean(mel_outputs_postnet[0,:,:341].cpu() - z.cpu())

tensor(0.3455)